<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# <center>Getting Started with the Arize Platform</center>
## <center> Investigating Embedding Drift in NLP: Sentiment Classification</center>

**In this walkthrough, we are going to ingest embedding data and look at embedding drift.**

In this scenario, you are in charge of maintaining a sentiment classification model. This simple model takes online reviews of your U.S.-based product as the input and predict whether the reviewer's sentiment was positive, negative, or neutral. You trained your sentiment classification model on English reviews. However, once the model was released into production, you notice that the performance of the model has degraded over a period of time.

Arize is able to surface the reason for this performance degradation. In this example, the presence of reviews written in Spanish impact the model's performance. You can surface and troubleshoot this issue by analyzing the _embedding vectors_ associated with the online review text.

It is worth noting that, according to our research, inspecting embedding drift can surface problems with your data before they cause performance degradation.

In this tutorial, we will start from scratch. We will:
* Download the data
* Preprocess the data
* Train the model
* Extract embedding vectors and predictions
* Log the inferences into the Arize Platform
* Visually explore embeddings in the Arize Platform

We will be using [🤗 Hugging Face](https://huggingface.co/)'s open source libraries to make this process extremely easy. In particular, we will use:
* [🤗 Datasets](https://huggingface.co/docs/datasets/index): a library used for easily accessing and sharing datasets, and evaluation metrics for Natural Language Processing (NLP), computer vision, and audio tasks.
* [🤗 Transformers](https://huggingface.co/docs/transformers/index): a library used to easily download and use state-of-the-art pre-trained models. Using pre-trained models can lower your compute costs, reduce your carbon footprint, and save you time from training a model from scratch.

**Note**: This example compares training vs production data. Arize supports sending only one dataset.

Let's get started!

# Step 0. Setup and Getting the Data

We will first install 🤗Hugging Face's `datasets` and `transformers` libraries, mentioned above. In addition, we will import some metrics from `scikit-learn`.

We'll explain each of the imports below as we use them through this tutorial.


## Install Dependencies and Import Libraries 📚

In [ ]:
!pip install -q datasets==2.9.0 transformers==4.26.1 arize

import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

from datetime import datetime
import uuid
from arize.pandas.logger import Client
from arize.utils.types import Environments, ModelTypes, EmbeddingColumnNames, Schema

## Check if GPU is available
Here we use Pytorch to check whether a GPU is available or not. When appropriate we will use PyTorch's `nn.Module.to()` method to ensure that the model will run on the GPU if we have one.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## **🌐 Download the Data**

The easiest way to load a dataset is from the [Hugging Face Hub](https://huggingface.co/datasets). There are already thousands of datasets in over 100 languages on the Hub. At Arize, we have crafted the [arize-ai/ecommerce_reviews_with_language_drift](https://huggingface.co/datasets/arize-ai/ecommerce_reviews_with_language_drift) dataset for this example notebook.

Thanks to Hugging Face 🤗 Datasets, we can download the dataset in one line of code. The `Dataset` object comes equipped with methods that make it very easy to inspect, pre-process, and post-process your data.

In [ ]:
dataset = load_dataset("arize-ai/ecommerce_reviews_with_language_drift")

You can select the splits of the dataset as you would in a dictionary.

In [ ]:
train_ds, val_ds, prod_ds = dataset['training'], dataset['validation'], dataset['production']

## Inspect the Data

It is often convenient to convert a `Dataset` object to a Pandas `DataFrame` so we can access high-level APIs for data visualization. 🤗 Datasets provides a `set_format()` method that allows us to change the output format of the `Dataset`. This does not change the underlying data format, an Arrow table. When the `DataFrame` format is no longer needed, we can reset the output format using `reset_format()`.

In [ ]:
train_ds.set_format("pandas")
display(train_ds[:].head())
train_ds.reset_format()

# Step 1. Developing your Sentiment Classification Model

## Pre-processing the data

Before being able to input the data into the model for fine-tuning we need to perform an important step: *__tokenization__*.

### Tokenization

Transformer models like __*DistilBERT*__ cannot receive raw strings as input. We need to _tokenize_ and _encode_ the text as numerical vectors. We will perform _Subword Tokenization_, which is learned from the pre-training corpus. Its goal is to allow tokenization of complex words (or misspellings) into smaller units that the model can learn, and keep common words as unique entities, keeping the length of the input to a reasonable size.

🤗 Transformers provides the `AutoTokenizer` class, which allows us to quickly download the tokenizer required by the pre-trained model of our choosing.

In this case, we will use the following __checkpoint__: `distilbert-base-uncased`.

In [ ]:
model_ckpt = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Next, let's define a processing function to tokenize the examples in the dataset. The `padding` and `truncation` options are added to keep the inputs to a consistent length. Shorter sequences are _padded_ and longer ones are _truncated_.

In [ ]:
def tokenize(batch, max_length=512):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=max_length)

### Apply to dataset

Now we apply the transformations above to the entire dataset using the `map()` method.

In [ ]:
process_batch_size = 100
max_size = 100 # max length of the tokenized sequence

train_ds = train_ds.map(lambda batch: tokenize(batch, max_size), batched=True, batch_size=process_batch_size)
val_ds = val_ds.map(lambda batch: tokenize(batch, max_size), batched=True, batch_size=process_batch_size)
prod_ds = prod_ds.map(lambda batch: tokenize(batch, max_size), batched=True, batch_size=process_batch_size)

In the following view of the dataset, two columns have appeared:
* `input_ids`: A numerical identifier to which each token has been mapped to.
* `attention_mask`: Array of 1s and 0s, allowing the model to ignore the padded parts of the inputs.

In [ ]:
train_ds.set_format("pandas")
display(train_ds[:].head())
train_ds.reset_format()

## Build the model

Similar to how we obtained the tokenizer, 🤗 Transformers provides the `AutoModelForSequenceClassification` class, which allows us to quickly download a pre-trained transformer model with a classification [task head](https://huggingface.co/course/en/chapter2/2?fw=pt#model-heads-making-sense-out-of-numbers) on top. The pre-trained model to use in this tutorial is [DistilBERT](https://huggingface.co/distilbert-base-uncased). The weights of the classification task head will be randomly initialized.

It is important to pass `output_hidden_states = True` to be able to compute the embedding vectors associated with the text (explained below).

_NOTE_: You can choose between fine-tuning the model in section A) or using a model that Arize has already fine-tuned for you in section B). To skip section A), set `SKIP_TRAINING = True` and go ahead to [_B) Download the model_](#B\)-Download-the-fine-tuned-model).

In [ ]:
model_name = f"distilbert_reviews_with_language_drift"
num_labels = dataset['training'].features['label'].num_classes

SKIP_TRAINING = False # Make True if you want to skip training

### A) Fine-tune the model

You may require Google Colab Pro or Pro+ to fine-tune the model in section A). If you want to avoid the cost, you can skip section A) and go to [_B) Download the model_](#B\)-Download-the-fine-tuned-model).

Let's download the pre-trained model,

In [ ]:
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels = num_labels, output_hidden_states=True)
         .to(device))

Further, we use the `TrainingArugments` class to define the training parameters. This class stores a lot of information and gives you control over the training and evaluation.

In [ ]:
training_batch_size = 16
training_epochs = 3
logging_steps = len(train_ds) // training_batch_size

training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=training_epochs,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=training_batch_size,
                                  per_device_eval_batch_size=training_batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim="adamw_torch"
                                  )

Define a metrics calculation function to evaluate the model.

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions[0].argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

Finally, we can fine-tune the model using the `Trainer` class.

In [ ]:
if SKIP_TRAINING == False:
    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
    )

    print("Evaluation before training")
    eval = trainer.evaluate(eval_dataset=val_ds)
    eval_df = pd.DataFrame({'Epoch':0, 'Validation Loss': eval['eval_loss'], 'Accuracy': eval['eval_accuracy'], 'F1': eval['eval_f1']}, index=[0])
    display(eval_df)
    print(" ")

    torch.cuda.empty_cache() # Free up some memory

    print("\nTraining...")
    trainer.train()

### B) Download the fine-tuned model

If you decided to skip step 1, you can download the already fine-tuned model [arize-ai/distilbert_reviews_with_language_drift](https://huggingface.co/arize-ai/distilbert_reviews_with_language_drift) from Arize's page in the Hugging Face Hub.

In [ ]:
if SKIP_TRAINING == True: # Make sure you marked SKIP_TRAINING = True if you wanted to skip training
    model_ckpt = f"arize-ai/{model_name}"

    model = (AutoModelForSequenceClassification
            .from_pretrained(model_ckpt, num_labels = num_labels, output_hidden_states=True)
            .to(device))

# Step 2. Post-Processing your data

In this case we will extract the prediction labels and the text embedding vectors. The latter are formed from the hidden states of the pre-trained (and then fine-tuned) model.

In [ ]:
def postprocess(batch, embedding_extraction_method="cls-token-last-layer", last_N=None):
    """
    Postprocessing includes:
    - Extract prediction labels
    - Extract embedding vectors associated with the input text
    """
    inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        out = model(**inputs)
        # Extract prediction labels #
        # ------------------------- #
        pred_label = out.logits.argmax(dim=1)
        # Extract embedding vectors #
        # ------------------------- #
        hidden_states = torch.stack(out.hidden_states)  # (layer_#, batch_size, seq_length/or/num_tokens, hidden_size)
        if embedding_extraction_method == "cls-token-last-layer":
            embeddings = hidden_states[-1][:,0,:] # Select last layer, then CLS token vector
        if embedding_extraction_method == "cls-token-avg-last-N-layers":
            if last_N <= 1:
                raise ValueError(f"last_N must be greater than 1, got {last_N}. If you want the last layer, use `cls-last`")
            embeddings = torch.mean(hidden_states[-last_N:], 0)[:,0,:]
        if embedding_extraction_method == "avg-all-tokens-last-layer":
            embeddings = torch.mean(hidden_states[-1], 1)
        if embedding_extraction_method == "avg-all-tokens-avg-last-N-layers":
            if last_N <= 1:
                raise ValueError(f"last_N must be greater than 1, got {last_N}. If you want the last layer, use `cls-last`")
            embeddings = torch.mean(torch.mean(hidden_states[-last_N:], 0), 1)

    return {"text_vector": embeddings.cpu().numpy(), "pred_label": pred_label.cpu().numpy()}

In [ ]:
train_ds.set_format("torch", columns=["input_ids", "attention_mask"])
train_ds = train_ds.map(postprocess, batched=True, batch_size=process_batch_size)

val_ds.set_format("torch", columns=["input_ids", "attention_mask"])
val_ds = val_ds.map(postprocess, batched=True, batch_size=process_batch_size)

prod_ds.set_format("torch", columns=["input_ids", "attention_mask"])
prod_ds = prod_ds.map(postprocess, batched=True, batch_size=process_batch_size)

# Step 3. Prepare your data to be sent to Arize

From this point forward, it is convenient to use Pandas DataFrames. We can do so easily using the format methods we have seen already

In [ ]:
train_df = train_ds.to_pandas()
val_df = val_ds.to_pandas()
prod_df = prod_ds.to_pandas()

## Update the timestamps

The data that you are working with was constructed in April of 2022. Hence, we will update the timestamps so they are current at the time that you're sending data to Arize.

In [ ]:
last_ts = max(prod_df['prediction_ts'])
now_ts = datetime.timestamp(datetime.now())
delta_ts = now_ts - last_ts

train_df['prediction_ts'] = (train_df['prediction_ts'] + delta_ts).astype(float)
val_df['prediction_ts'] = (val_df['prediction_ts'] + delta_ts).astype(float)
prod_df['prediction_ts'] = (prod_df['prediction_ts'] + delta_ts).astype(float)

## Map labels to class names

We want to log the inferences with the corresponding class names (for predictions and actuals) instead of the numeric label. Since we used 🤗 Datasets to download the dataset, it came equipped with methods to do this.

The dataset we downloaded defined the label to be an instance of the `datasets.ClassLabel` class, which has the convenient method `int2str` (visit [Hugging Face documentation](https://huggingface.co/docs/datasets/v2.2.1/en/package_reference/main_classes#datasets.ClassLabel.names) for more information).

In [ ]:
def label_int2str(row):
    return train_ds.features['label'].int2str(row)

In [ ]:
train_df['label'] = train_df['label'].apply(label_int2str)
val_df['label'] = val_df['label'].apply(label_int2str)
prod_df['label'] = prod_df['label'].apply(label_int2str)

train_df['pred_label'] = train_df['pred_label'].apply(label_int2str)
val_df['pred_label'] = val_df['pred_label'].apply(label_int2str)
prod_df['pred_label'] = prod_df['pred_label'].apply(label_int2str)

## Add prediction ids

The Arize platform uses prediction IDs to link a prediction to an actual. Visit the [Arize documentation](https://docs.arize.com/arize/data-ingestion/model-schema/5.-prediction-id?q=prediction_id) for more details.

You can generate prediction IDs as follows:

In [ ]:
def add_prediction_id(df):
    return [str(uuid.uuid4()) for _ in range(df.shape[0])]

In [ ]:
train_df['prediction_id'] = add_prediction_id(train_df)
val_df['prediction_id'] = add_prediction_id(val_df)
prod_df['prediction_id'] = add_prediction_id(prod_df)

# Step 4. Sending Data into Arize 💫

## Select the columns we want to send to Arize (optional)

This step is not really necessary, since we will select the columns we want to send to Arize using the `Schema` definition (below). However, for the purpose of visibility, this is our final `DataFrame` with the data that will be sent to Arize.

In [ ]:
arize_columns = [
    'prediction_id',
    'prediction_ts',
    'reviewer_age',
    'reviewer_gender',
    'product_category',
    'language',
    'text',
    'text_vector',
    'label',
    'pred_label'
    ]

train_df = train_df[arize_columns]
val_df = val_df[arize_columns]
prod_df = prod_df[arize_columns]

train_df.head()

## Import and Setup Arize Client

The first step is to setup the Arize client. After that we will log the data.

Copy the Arize `API_KEY` and `SPACE_ID` from your Space Settings page (shown below) to the variables in the cell below. We will also be setting up some metadata to use across all logging.

<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-id-and-key.png" width="700">

In [ ]:
SPACE_ID = "SPACE_ID"
API_KEY = "API_KEY"
arize_client = Client(space_id=SPACE_ID, api_key=API_KEY)
model_id = "NLP-demo-hf-sentiment-classification-language-drift"
model_version = "1.0"
model_type = ModelTypes.SCORE_CATEGORICAL
if SPACE_ID == "SPACE_ID" or API_KEY == "API_KEY":
    raise ValueError("❌ CHANGE SPACE_ID AND/OR API_KEY")
else:
    print("✅ Import and Setup Arize Client Done! Now we can start using Arize!")


Now that our Arize client is setup, let's go ahead and log all of our data to the platform. For more details on how **`arize.pandas.logger`** works, visit our documentation.

[![Buttons_OpenOrange.png](https://storage.googleapis.com/arize-assets/fixtures/Buttons_OpenOrange.png)](https://docs.arize.com/arize/sdks-and-integrations/python-sdk/arize.pandas)

## Define the Schema

A Schema instance specifies the column names for corresponding data in the dataframe. While we could define different Schemas for training and production datasets, the dataframes have the same column names, so the Schema will be the same in this instance.

To ingest non-embedding features, it suffices to provide a list of column names that contain the features in our dataframe. Embedding features, however, are a little bit different.

Arize allows you to ingest not only the embedding vector, but the raw data associated with that embedding, or a URL link to that raw data. Therefore, up to 3 columns can be associated to the same _embedding object_*. To be able to do this, Arize's SDK provides the `EmbeddingColumnNames` class, used below.

*NOTE: This is how we refer to the 3 possible pieces of information that can be sent as embedding objects:
* Embedding `vector` (required)
* Embedding `data` (optional): raw text associated with the embedding vector
* Embedding `link_to_data` (optional): link to the data file (image, audio, ...) associated with the embedding vector

Learn more [here](https://docs.arize.com/arize/sending-data/model-schema-reference#8.-embedding-features-unstructured).

In [ ]:
features = [
    'reviewer_age',
    'reviewer_gender',
    'product_category',
    'language',
]

embedding_features = {
    # Dictionary keys will be the displayed name of the embedding feature in the app
    "text_embedding": EmbeddingColumnNames(
        vector_column_name="text_vector",
        data_column_name="text",
    ),
}

# Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="pred_label",
    actual_label_column_name="label",
    feature_column_names=features,
    embedding_feature_column_names=embedding_features
)

## Log Training Data

**Note**: This example compares training vs production data. Arize supports sending only one dataset.

In [ ]:
# Logging Training DataFrame
response = arize_client.log(
    dataframe=train_df,
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    environment=Environments.TRAINING,
    schema=schema
)


# If successful, the server will return a status_code of 200
if response.status_code != 200:
    print(f"❌ logging failed with response code {response.status_code}, {response.text}")
else:
    print(f"✅ You have successfully logged training set to Arize")


## Log Validation Data

In [ ]:
# Logging Validation DataFrame
response = arize_client.log(
    dataframe=val_df,
    model_id=model_id,
    model_version=model_version,
    batch_id="validation",
    model_type=model_type,
    environment=Environments.VALIDATION,
    schema=schema
)


# If successful, the server will return a status_code of 200
if response.status_code != 200:
    print(f"❌ logging failed with response code {response.status_code}, {response.text}")
else:
    print(f"✅ You have successfully logged training set to Arize")


## Log Production Data

In [ ]:
# Logging Production DataFrame
response = arize_client.log(
    dataframe=prod_df,
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    environment=Environments.PRODUCTION,
    schema=schema
)

if response.status_code != 200:
    print(f"❌ logging failed with response code {response.status_code}, {response.text}")
else:
    print(f"✅ You have successfully logged production set to Arize")

# Step 5. Confirm Data in Arize ✅
Note that the Arize platform takes about 15 minutes to index embedding data. While the model should appear immediately, the data will not show up until the indexing is complete. Feel free to head over to the **Data Ingestion** tab for your model to watch Arize work its magic!🔮

You will be able to see the predictions, actuals, and feature importances that have been sent in the last 30 minutes, last day or last week.

An example view of the Data Ingestion tab from a model, when data is sent continuously over 30 minutes, is shown in the image below.

<img src="https://storage.googleapis.com/arize-assets/fixtures/data-ingestion-tab.png" width="700">

# Check the Embedding Data in Arize

First, set the baseline to the training set that we logged before.

<img src="https://storage.cloud.google.com/arize-assets/fixtures/embedding_setup_baseline.gif" width="700">


If your model contains embedding data, you will see it in your Model's Overview page.

<img src="https://storage.cloud.google.com/arize-assets/fixtures/Embeddings/NLP/NLP-reviews-demo-language-drift-overview.jpg" width="700">

 Click on the Embedding Name or the Euclidean Distance value to see how your embedding data is drifting over time. In the picture below we represent the global euclidean distance between your production set (at different points in time) and the baseline (which we set to be our training set). We can see there is a period of a week where suddenly the distance is remarkably higher. This shows us that during that time text data was sent to our model that was different than what it was trained on (English). This is the period of time when reviews written in Spanish were sent alongside the expected English reviews.

<img src="https://storage.cloud.google.com/arize-assets/fixtures/Embeddings/NLP/NLP-reviews-demo-language-drift-emb-0.jpg" width="700">

In addition to the drift tracking plot above, below you can find the UMAP visualization of your data, according to the point in time selected. Notice that the production data and our baseline (training) data are superimposed, which is indicative that the model is seeing data in production similar to the data it was trained on.

<img src="https://storage.cloud.google.com/arize-assets/fixtures/Embeddings/NLP/NLP-reviews-demo-language-drift-emb-1.jpg" width="700">

Next, select a point in time when the drift was high and select a UMAP visualization in 2D. We can see that both training and production data are superimposed for the most part, but another cluster of production data has appeared. This indicates that the model is seeing data in production qualitatively different to the data it was trained on, and in this case causing performance degradation.

<img src="https://storage.cloud.google.com/arize-assets/fixtures/Embeddings/NLP/NLP-reviews-demo-language-drift-emb-2.jpg" width="700">

For further inspection, you may select a 3D UMAP view and clicked _Explore UMAP_ to expand the view. With this view we can interact in 3D with our dataset. We can zoom, rotate, and drag so we can see the areas of our dataset that are most interesting to us. Check out the workflow below:

<img src="https://storage.cloud.google.com/arize-assets/fixtures/Embeddings/NLP/NLP-reviews-demo-language-drift-workflow.gif" width="700">

In the display above, Arize offers many coloring options:
1. By Dataset: You can see that the coloring has been made to distinguish production data vs baseline data (training in this example). This is specifically useful to detect drift. In this example, we can see that there is some production data far away from any training data, giving an indication of severe dataset drift. We can identify exactly what datapoints our baseline is missing so that re-train effectively.
2. By Prediction Label: This coloring option gives an insight on how is our model making decisions. Where are the different classes located in the space? Is the model predicting one class in regions where it should be predicting another?
3. By Actual Label: This coloring option is great if we want to identify labeling issues. For instance, if inside the orange cloud, we can see points of other colors, it is a good idea to check and see if the labels are wrong. Further we can use the corrected labels for re-training.
4. By Correctness: This coloring option offers a quick way of identifying where the bulk of your model's mistakes are placed, giving you an area to pay attention to. In this example, we can see that the spanish reviews are almost all red.
5. By Confusion Matrix: This coloring option allows you to select a `positive class` and color the data-points as `True Positives`, `True Negatives`, `False Positives`, `False Negatives`.
6. By Feature: You can identify areas of the space where your model might be underperforming and, by coloring the points by feature, identify patterns at feature level. In other words, you can identify a slice of your data sharing a common feature (or features) that are causing a problem.
7. By Prediction Score: You can identify areas where your model is more confident of its predictions and areas where your model struggled more to make a decision.

More coloring options will be added to help you understand and debug your model and dataset.

# Wrap Up 🎁
Congratulations, you've now sent your first machine learning embedding data to the Arize platform!!

Additionally, if you want to remove this example model from your account, just click **Models** -> **NLP-reviews-demo-language-drift** -> **config** -> **delete**

### Overview
Arize is an end-to-end ML observability and model monitoring platform. The platform is designed to help ML engineers and data science practitioners surface and fix issues with ML models in production faster with:
- Automated ML monitoring and model monitoring
- Workflows to troubleshoot model performance
- Real-time visualizations for model performance monitoring, data quality monitoring, and drift monitoring
- Model prediction cohort analysis
- Pre-deployment model validation
- Integrated model explainability

### Website
Visit Us At: https://arize.com/model-monitoring/

### Additional Resources
- [What is ML observability?](https://arize.com/what-is-ml-observability/)
- [Monitor Unstructured Data with Arize](https://arize.com/blog/monitor-unstructured-data-with-arize)
- [Getting Started With Embeddings Is Easier Than You Think](https://arize.com/blog/getting-started-with-embeddings-is-easier-than-you-think)
- [Playbook to model monitoring in production](https://arize.com/the-playbook-to-monitor-your-models-performance-in-production/)
- [Using statistical distance metrics for ML monitoring and observability](https://arize.com/using-statistical-distance-metrics-for-machine-learning-observability/)
<!-- - [ML infrastructure tools for data preparation](https://arize.com/ml-infrastructure-tools-for-data-preparation/) -->
- [ML infrastructure tools for model building](https://arize.com/ml-infrastructure-tools-for-model-building/)
- [ML infrastructure tools for production](https://arize.com/ml-infrastructure-tools-for-production-part-1/)
<!-- - [ML infrastructure tools for model deployment and model serving](https://arize.com/ml-infrastructure-tools-for-production-part-2-model-deployment-and-serving/) -->

- [ML infrastructure tools for ML monitoring and observability](https://arize.com/ml-infrastructure-tools-ml-observability/)

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.
